
# Visium-Clustergrammer2



In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
# df = {}

from clustergrammer2 import net

import ipywidgets as widgets
import numpy as np
from bqplot import pyplot as plt
import bqplot
from ipywidgets import HBox

from copy import deepcopy
from glob import glob
from scipy.spatial.distance import pdist, squareform
from scipy.spatial import Voronoi

import warnings
warnings.filterwarnings('ignore')

>> clustergrammer2 backend version 0.5.6


In [3]:
df_meta = pd.read_csv('data/processed/df_meta.csv', index_col=0)
df_meta.head()

,gex-umi-sum,num_expressed_genes,gex-ribo-avg,gex-mito-avg,in_tissue,array_row,array_col,pxl_col_in_fullres,pxl_row_in_fullres,pos-x,pos-y,gex-umi-sum-norm,Cell Type
AAACAAGTATCTCCCA,13069,4242,7.380952,97.600000,1,50,102,7475,8501,8501,2994,0.194714,Pvalb
AAACACCAATAACTGC,37448,7860,19.809524,256.666667,1,59,19,8553,2788,2788,1916,0.557934,Pvalb
AAACAGAGCGACTCCT,28475,6332,15.123810,275.533333,1,14,94,3164,7950,7950,7305,0.424246,L2/3
AAACAGCTTTCAGAAG,39718,7957,27.676190,184.400000,1,43,9,6637,2099,2099,3832,0.591755,Vip
AAACAGGGTCTATATT,33392,7791,23.447619,117.600000,1,47,13,7116,2375,2375,3353,0.497504,Vip


In [4]:
df_meta.shape

(2696, 13)

In [5]:
net.viz = net.load_json_to_dict('data/processed/pre-calc-viz.json')

In [6]:
ini_cat_colors = net.viz['cat_colors']['col']['cat-0']
cat_colors = {}
for inst_key in ini_cat_colors:
    cat_colors[inst_key.split(': ')[1]] = ini_cat_colors[inst_key]

In [7]:
# df = pd.read_csv('data/processed/df_meta.csv', index_col=0)
# df.shape

In [9]:
df_meta['plot-x'] = df_meta['pos-x'].apply(lambda x: x/2 - 700)
df_meta['plot-y'] = df_meta['pos-y'].apply(lambda x: x/2 )

In [10]:
df_gex = pd.read_csv('data/processed/gex.csv', index_col=0)
df_gex.shape

(250, 2696)

In [11]:
df_meta.columns.tolist()

['gex-umi-sum',
 'num_expressed_genes',
 'gex-ribo-avg',
 'gex-mito-avg',
 'in_tissue',
 'array_row',
 'array_col',
 'pxl_col_in_fullres',
 'pxl_row_in_fullres',
 'pos-x',
 'pos-y',
 'gex-umi-sum-norm',
 'Cell Type',
 'plot-x',
 'plot-y']

In [13]:
x_dim = 2000
y_dim = 2000

fig = plt.figure()

color_list = [ cat_colors[df_meta.loc[x, 'Cell Type']] for x in df_gex.columns.tolist()]

def_tt = bqplot.Tooltip(fields=['name', 'Cell Type'], formats=[''])


scatter = plt.scatter(df_meta['plot-x'], 
                      df_meta['plot-y'],
                      display_names=False, 
                      default_size=20, 
                      tooltip=def_tt,
                      colors=color_list)


scatter.default_size = 100
scatter.default_opacities = [0.5]*5164

top_margin = 200
inst_width = 800
inst_height = 1000
fig.layout.min_height = str(inst_height) + 'px'
fig.layout.min_width  = str(inst_width) + 'px'

plt.xlim(0, 2.0*x_dim)
plt.ylim(0, 2.0*y_dim)
fig.title = 'Visium Tissue'
fig.fig_margin = {'top': top_margin, 'bottom': 5, 'left': 5, 'right': 5}


In [14]:
# df['gex-umi-sum'].tolist()

In [15]:
default_opacity = 0.75

In [16]:
def cat_highlight(inst_value):
    
    cols = df_gex.columns.tolist()

    if inst_value == 'reset_cats':
        list_opacities = [default_opacity for x in cols]
        scatter.default_opacities = list_opacities
        
    else:
        
        inst_cat_title = inst_value.split(': ')[0]
        inst_cat = inst_value.split(': ')[1]

        list_opacities = []

        for inst_label in cols:
            inst_opacity = 0.15
            if df_meta.loc[inst_label, inst_cat_title] == inst_cat:
                inst_opacity = 1
            list_opacities.append(inst_opacity)        

    # patch.opacities = list_opacities
    scatter.default_opacities = list_opacities

In [17]:
def on_value_change(change):
    
    
    if change['new'] == 'null':
        cat_highlight('reset_cats')
        
        fig.title = 'Visium Tissue'
    else: 
        # mousing over category
        if 'cat-' in change['new']:
            inst_cat = change['new'].split(' -> ')[1]
            
            # print('on_value_change', inst_cat)
            cat_highlight(inst_cat)       
            
            fig.title = 'Visium Tissue'
            
        # mousing over marker
        else:
            inst_marker = change['new'].split(' -> ')[1]
            ser_opacity = df_gex.loc[inst_marker]
            
            fig.title = 'Visium Tissue: ' + inst_marker
            
            
            # the scatterplot dots are not in the same order as the data in the dataframe
            #list_opacities = [float(x/ser_opacity.max()) for x in list(ser_opacity.get_values())]
            
            # loop up opacities
            list_opacities = []
            rows = ser_opacity.index.tolist()
            for inst_name in df_gex.columns.tolist():
                if inst_name in rows:
                    list_opacities.append( float(ser_opacity[inst_name]/ser_opacity.max())  )
                else:
                    list_opacities.append(0)
                
            scatter.default_opacities = list_opacities


In [18]:
net.widget()
net.widget_instance.observe(on_value_change, names='value')

In [19]:
HBox([fig, net.widget_instance])